In [3]:
!pip install numpy librosa torch scikit-learn torchvision transformers pysoundfile SoundFile
!pip install --force-reinstall soundfile


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached soundfile-0.12.1-py2.py3-none-macosx_11_0_arm64.whl (1.1 MB)
  Obtaining dependency information for cffi>=1.0 from https://files.pythonhosted.org/packages/18/6c/0406611f3d5aadf4c5b08f6c095d874aed8dfc2d3a19892707d72536d5dc/cffi-1.16.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached cffi-1.16.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (1.5 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
Using cached cffi-1.16.0-cp311-cp311-macosx_11_0_arm64.whl (176 kB)
  Attempting uninstall: pycparser
    Found existing installation: pycparser 2.21
    Uninstalling pycparser-2.21:
      Successfully uninstalled pycparser-2.21
  Attempting uninstall: cffi
    Found existing installation: cffi 1.16.0
    Uninstalling cffi-1.16.0:
      Successfully uninstalled cffi-1.16.0
  Attempting uninstall: soundfile
    Found existing installation: soundfile 0.1

In [4]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import ClapModel, ClapProcessor
from scipy.io import wavfile
import soundfile as sf

In [5]:
# Initialize CLAP model
model = ClapModel.from_pretrained('laion/larger_clap_music')
processor = ClapProcessor.from_pretrained('laion/larger_clap_music')


In [6]:
# Function to load and process a single .wav file
def load_and_resample(file_path, target_sr=48000):
    try:
        # Load the audio file
        audio, sr = librosa.load(file_path, sr=None)  # Load with original sample rate
        # Resample the audio to the target sample rate
        if sr != target_sr:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)
        return audio, target_sr
    except Exception as e:
        # Handle exceptions (corrupt or unreadable files)
        print(f"Error loading {file_path}: {e}")
        return None, None

# Path to the dataset
dataset_path = 'data'

# Processed data
processed_data = []
processed_labels = []

# Iterate over genres and files
for genre in os.listdir(dataset_path):
    genre_path = os.path.join(dataset_path, genre)
    if genre == '.DS_Store':
        continue
    for file in os.listdir(genre_path):
        if file.endswith('.wav'):
            file_path = os.path.join(genre_path, file)
            audio, sr = load_and_resample(file_path)
            if audio is not None:
                processed_data.append({'array': audio, 'sampling_rate': sr})
                processed_labels.append(genre)

/var/folders/dl/yt789zmj6vj9qrgvvg0vbj0h0000gn/T/ipykernel_55995/2646419869.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=None)  # Load with original sample rate
/Users/diptobiswas/.pyenv/versions/3.11.6/lib/python3.11/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error loading data/jazz/jazz.00054.wav: 


In [ ]:
# Split the dataset
train, test = train_test_split(processed_data, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Function to get embeddings
def get_embeddings(dataset):
    embeddings = []
    for sample in dataset:
        inputs = processor(audios=sample['array'], sampling_rate=sample['sampling_rate'], return_tensors='pt')
        embeddings.append(model.get_audio_features(**inputs))
    return embeddings

# Get embeddings for each set
train_embeddings = get_embeddings(train)
val_embeddings = get_embeddings(val)
test_embeddings = get_embeddings(test)